In [1]:
from lorahub.algorithm import lorahub_inference
import os
import json
from lorahub.algorithm2 import lorahub_learning, lorahub_inference
import lorahub.algorithm2 as algorithm2
import lorahub.algorithm3_grad as algorithm3_grad
from lorahub.constant import LORA_MODULE_NAMES
import random
from random import shuffle
import sys
import pandas as pd

In [2]:
def evaluate_flan_results_zero_shot(folder, flan_model_name):
    sub_dirs = os.listdir(folder)
    result={}
    for sub_dir in sub_dirs:
        test_file_path = os.path.join(folder, sub_dir, "zero_shot.jsonl")
        task_inputs, task_outputs = [], []
        for line in open(test_file_path, "r", encoding="utf-8"):
            example = json.loads(line)
            task_inputs.append(example["context"])
            task_outputs.append(example["completion"])
        print("Evaluating on task (zero shot): ", sub_dir)
        _,task_acc=lorahub_inference(task_inputs,
                          flan_model_name,
                          flan_model_name,
                          16,
                          task_outputs)
        print("task accuracy:",task_acc)
        # lorahub_inference(task_inputs,
        #                   flan_model_name,
        #                   flan_model_name,
        #                   16,
        #                   task_outputs)
        result[sub_dir]=task_acc
    result_pd=pd.DataFrame({'Zero-shot acc':result})
    return result,result_pd
        # break


def evaluate_flan_results_few_shot(folder, flan_model_name):
    sub_dirs = os.listdir(folder)
    result={}
    for sub_dir in sub_dirs:
        test_file_path = os.path.join(folder, sub_dir, "few_shot.jsonl")
        task_inputs, task_outputs = [], []
        for line in open(test_file_path, "r", encoding="utf-8"):
            example = json.loads(line)
            task_inputs.append(example["context"])
            task_outputs.append(example["completion"])
        print("Evaluating on task (five shot): ", sub_dir)
        _,task_acc=lorahub_inference(task_inputs,
                          flan_model_name,
                          flan_model_name,
                          16,
                          task_outputs)
        result[sub_dir]=task_acc
    result_pd=pd.DataFrame({'Few-shot acc':result})
    return result,result_pd

7
def evaluate_lorahub_results_few_shot(folder, flan_model_name):
    sub_dirs = os.listdir(folder)
    result={'lorahub avg acc':{},'lorahub max acc':{}}
    # 5 seeds used in our experiments
    for sub_dir in sub_dirs:
        print("Evaluating on task (lorahub): ", sub_dir)
        # construct the few-shot examples for lorahub learning
        example_inputs, examples_outputs = [], []
        example_file_path = os.path.join(folder, sub_dir, "example.jsonl")
        for line in open(example_file_path, "r", encoding="utf-8"):
            example = json.loads(line)
            example_inputs.append(example["context"])
            examples_outputs.append(example["completion"])
            
        # random select 5 examples for each task
        random.seed(42)
        shuffled_set = list(zip(example_inputs, examples_outputs))
        random.shuffle(shuffled_set)
        example_inputs, examples_outputs = zip(*shuffled_set)
        # take the first 5 examples
        example_inputs, examples_outputs = example_inputs[:5], examples_outputs[:5]

        # load the zero-shot examples for evaluation
        test_file_path = os.path.join(folder, sub_dir, "zero_shot.jsonl")
        task_inputs, task_outputs = [], []
        for line in open(test_file_path, "r", encoding="utf-8"):
            example = json.loads(line)
            task_inputs.append(example["context"])
            task_outputs.append(example["completion"])

        task_perf_list = []
        for seed in range(1, 6):
            random.seed(seed)
            print("Evaluating on task (lorahub): ", sub_dir, "with seed:", seed)

            def get_lora_module_list():
                return random.sample(LORA_MODULE_NAMES, 20) #what 
            # get a list of modules to be used in the composition
            modules = get_lora_module_list()

            # perform LoRAHub learning
            module_weights, model, tokenizer = lorahub_learning(lora_module_list=modules,
                                                                example_inputs=example_inputs,
                                                                example_outputs=examples_outputs,
                                                                max_inference_step=40,
                                                                batch_size=5)

            print("module_weights:", module_weights)

            """
            Perform inference to get predictions
            """
            _, task_acc = lorahub_inference(example_inputs=task_inputs,
                                            model_or_name_path=model,
                                            tokenizer_or_tokenizer_path=tokenizer,
                                            batch_size=10,
                                            # can set as None if you do not have the ground truth
                                            example_outputs=task_outputs)
            task_perf_list.append(task_acc)
            break
        avg_perf, max_perf = sum(task_perf_list) / len(task_perf_list), max(task_perf_list)
        print("average perf:", avg_perf, "best perf:", max_perf)
        result["lorahub avg acc"][sub_dir]=avg_perf
        result["lorahub max acc"][sub_dir]=max_perf
        break
    result_pd=pd.DataFrame(result)
    return result,result_pd

In [ ]:
# if not os.path.exists("data_bbh"):
#     # download dataset
#     os.system("wget https://github.com/sail-sg/lorahub/releases/download/0.1/data_bbh.zip")
#     # unzip
#     os.system("unzip data_bbh.zip")
# evaluate the model
result_folder = "results"
zero_result,zero_result_df=evaluate_flan_results_zero_shot("data_bbh", "google/flan-t5-large")
zero_result_df.to_csv(os.path.join(result_folder, "zero_result.csv"))

# five shot for flan models
few_result,few_result_df=evaluate_flan_results_few_shot("data_bbh", "google/flan-t5-large")
few_result_df.to_csv(os.path.join(result_folder, "few_result.csv"))
# five shot for lorahub models
lorahub_result,lorahub_result_df=evaluate_lorahub_results_few_shot("data_bbh", "google/flan-t5-large")
lorahub_result_df.to_csv(os.path.join(result_folder, "lorahub_result.csv"))

#save all the results to a csv file
result_df=pd.DataFrame({'zero-shot acc':zero_result,'few-shot acc':few_result,'lorahub avg acc':lorahub_result['lorahub avg acc'],'lorahub max acc':lorahub_result['lorahub max acc']})
result_df.to_csv(os.path.join(result_folder, "result.csv"))


In [ ]:
lorahub_df = pd.read_csv('results/lorahub_result.csv',index_col=0).sort_index()
few_df = pd.read_csv('results/few_result.csv',index_col=0).sort_index()
zero_df = pd.read_csv('results/zero_result.csv',index_col=0).sort_index()
print(lorahub_df)
# Merge the data frames
merged_df = pd.concat([lorahub_df, few_df, zero_df], axis=1)

# # Display the merged dataframe
print(merged_df)

# Optionally, save the merged dataframe to a new CSV file
merged_df.to_csv('results/merged_result.csv', index=True)

In [2]:
folder = "data_bbh"
flan_model_name = "google/flan-t5-large"
sub_dirs = os.listdir(folder)
result={'lorahub avg acc':{},'lorahub max acc':{}}
sub_dir=sub_dirs[0]
# 5 seeds used in our experiments
print("Evaluating on task (lorahub): ", sub_dir)
# construct the few-shot examples for lorahub learning
example_inputs, examples_outputs = [], []
example_file_path = os.path.join(folder, sub_dir, "example.jsonl")
for line in open(example_file_path, "r", encoding="utf-8"):
    example = json.loads(line)
    example_inputs.append(example["context"])
    examples_outputs.append(example["completion"])
    
# random select 5 examples for each task
random.seed(42)
shuffled_set = list(zip(example_inputs, examples_outputs))
random.shuffle(shuffled_set)
example_inputs, examples_outputs = zip(*shuffled_set)
# take the first 5 examples
example_num=100
example_inputs, examples_outputs = example_inputs[:example_num], examples_outputs[:example_num]

# load the zero-shot examples for evaluation
test_file_path = os.path.join(folder, sub_dir, "zero_shot.jsonl")
task_inputs, task_outputs = [], []
for line in open(test_file_path, "r", encoding="utf-8"):
    example = json.loads(line)
    task_inputs.append(example["context"])
    task_outputs.append(example["completion"])


random.seed(42)
def get_lora_module_list():
        return random.sample(LORA_MODULE_NAMES, 100) #what 
    # get a list of modules to be used in the composition

modules = get_lora_module_list()



Evaluating on task (lorahub):  boolean_expressions


In [3]:
#reimport lorahub.algorithm2
import importlib
import lorahub.algorithm3_grad_old as algorithm3_grad_old
importlib.reload(algorithm3_grad)
importlib.reload(algorithm3_grad_old)
from lorahub.algorithm3_grad import lorahub_learning, lorahub_inference
# from lorahub.algorithm3_grad_old import lorahub_learning, lorahub_inference

In [4]:
task_perf_list = []
seed=1
print("Evaluating on task (lorahub): ", sub_dir, "with seed:", seed)



# perform LoRAHub learning
module_weights, model, tokenizer = lorahub_learning(lora_module_list=modules,
                                                    example_inputs=example_inputs,
                                                    example_outputs=examples_outputs,
                                                    max_inference_step=30,
                                                    batch_size=5,
                                                    early_stopping=True,
                                                    lr=0.02)

print("module_weights:", module_weights)

"""
Perform inference to get predictions
"""
_, task_acc = lorahub_inference(example_inputs=task_inputs,
                                model_or_name_path=model,
                                tokenizer_or_tokenizer_path=tokenizer,
                                batch_size=10,
                                 # can set as None if you do not have the ground truth
                                example_outputs=task_outputs)
task_perf_list.append(task_acc)
avg_perf, max_perf = sum(task_perf_list) / len(task_perf_list), max(task_perf_list)
print("average perf:", avg_perf, "best perf:", max_perf)
result["lorahub avg acc"][sub_dir]=avg_perf
result["lorahub max acc"][sub_dir]=max_perf

Evaluating on task (lorahub):  boolean_expressions with seed: 1
> Begin to load lora modules


  0%|          | 0/100 [00:00<?, ?it/s]

> Loading lorahub/flan_t5_large-glue_mrpc ...


  1%|          | 1/100 [00:00<00:55,  1.79it/s]

> Loading lorahub/flan_t5_large-dream_baseline ...


...

Running tokenizer on dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

num_blocks: 48
> Begin to perform gradient optimization ...
Step 0, loss 0.05648682203463977
Step 1, loss 0.037820473246392795
Step 2, loss 0.030990637662034713
Step 3, loss 0.022491906974755692
Step 4, loss 0.07842005029233405
Step 5, loss 0.013905840214283672
Step 6, loss 0.030855559828341938
Step 7, loss 0.015784347451699432
Step 8, loss 0.0074658355326391755
Step 9, loss 0.004477370080712717
Step 10, loss 0.0028234606550313402
Step 11, loss 0.010425313632867982
Step 12, loss 0.014777394551492762
Step 13, loss 0.007724030695226247
Step 14, loss 0.00542082952824785
Step 15, loss 0.0010344053202743452
Step 16, loss 0.0007996597053647747
Step 17, loss 0.00038478464636568786
Step 18, loss 0.00017636726213936526
Step 19, loss 0.00015627636638981812
Step 20, loss 0.0001497667541130454
Step 21, loss 0.0001458272687912654
Step 22, loss 0.00014287373603991683
Step 23, loss 0.0001405190570267223
Step 24, loss 0.00013860059200254682
Step 25, loss 0.0001369847688977188
Step 26, loss 0.000135604

Running tokenizer on dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

C:\Users\GJK\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:2778: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


average perf: 84.0 best perf: 84.0
